In [114]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
df = pd.read_csv('data/used_cars.csv')

In [115]:
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4009 entries, 0 to 4008
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   brand         4009 non-null   object
 1   model         4009 non-null   object
 2   model_year    4009 non-null   int64 
 3   milage        4009 non-null   object
 4   fuel_type     3839 non-null   object
 5   engine        4009 non-null   object
 6   transmission  4009 non-null   object
 7   ext_col       4009 non-null   object
 8   int_col       4009 non-null   object
 9   accident      3896 non-null   object
 10  clean_title   3413 non-null   object
 11  price         4009 non-null   object
dtypes: int64(1), object(11)
memory usage: 376.0+ KB


,brand,model,model_year,milage,fuel_type,engine,transmission,ext_col,int_col,accident,clean_title,price
0,Ford,Utility Police Interceptor Base,2013,"51,000 mi.",E85 Flex Fuel,300.0HP 3.7L V6 Cylinder Engine Flex Fuel Capa...,6-Speed A/T,Black,Black,At least 1 accident or damage reported,Yes,"$10,300"
1,Hyundai,Palisade SEL,2021,"34,742 mi.",Gasoline,3.8L V6 24V GDI DOHC,8-Speed Automatic,Moonlight Cloud,Gray,At least 1 accident or damage reported,Yes,"$38,005"
2,Lexus,RX 350 RX 350,2022,"22,372 mi.",Gasoline,3.5 Liter DOHC,Automatic,Blue,Black,None reported,NaN,"$54,598"
3,INFINITI,Q50 Hybrid Sport,2015,"88,900 mi.",Hybrid,354.0HP 3.5L V6 Cylinder Engine Gas/Electric H...,7-Speed A/T,Black,Black,None reported,Yes,"$15,500"
4,Audi,Q3 45 S line Premium Plus,2021,"9,835 mi.",Gasoline,2.0L I4 16V GDI DOHC Turbo,8-Speed Automatic,Glacier White Metallic,Black,None reported,NaN,"$34,999"


In [116]:
df.drop(['clean_title'], axis=1, inplace=True)

In [117]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4009 entries, 0 to 4008
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   brand         4009 non-null   object
 1   model         4009 non-null   object
 2   model_year    4009 non-null   int64 
 3   milage        4009 non-null   object
 4   fuel_type     3839 non-null   object
 5   engine        4009 non-null   object
 6   transmission  4009 non-null   object
 7   ext_col       4009 non-null   object
 8   int_col       4009 non-null   object
 9   accident      3896 non-null   object
 10  price         4009 non-null   object
dtypes: int64(1), object(10)
memory usage: 344.7+ KB


In [118]:
cols_cat = ['brand','model','milage','fuel_type','engine','transmission','ext_col','int_col','accident','price']

for col in cols_cat:
    print(f'Column {col}: {df[col].nunique()} subniveles')

Column brand: 57 subniveles
Column model: 1898 subniveles
Column milage: 2818 subniveles
Column fuel_type: 7 subniveles
Column engine: 1146 subniveles
Column transmission: 62 subniveles
Column ext_col: 319 subniveles
Column int_col: 156 subniveles
Column accident: 2 subniveles
Column price: 1569 subniveles


In [119]:
df.describe()

,model_year
count,4009.000000
mean,2015.515590
std,6.104816
min,1974.000000
25%,2012.000000
50%,2017.000000
75%,2020.000000
max,2024.000000


In [120]:
df.columns = df.columns.str.strip().str.lower().str.replace(' ', '_')

In [121]:
df.head()

,brand,model,model_year,milage,fuel_type,engine,transmission,ext_col,int_col,accident,price
0,Ford,Utility Police Interceptor Base,2013,"51,000 mi.",E85 Flex Fuel,300.0HP 3.7L V6 Cylinder Engine Flex Fuel Capa...,6-Speed A/T,Black,Black,At least 1 accident or damage reported,"$10,300"
1,Hyundai,Palisade SEL,2021,"34,742 mi.",Gasoline,3.8L V6 24V GDI DOHC,8-Speed Automatic,Moonlight Cloud,Gray,At least 1 accident or damage reported,"$38,005"
2,Lexus,RX 350 RX 350,2022,"22,372 mi.",Gasoline,3.5 Liter DOHC,Automatic,Blue,Black,None reported,"$54,598"
3,INFINITI,Q50 Hybrid Sport,2015,"88,900 mi.",Hybrid,354.0HP 3.5L V6 Cylinder Engine Gas/Electric H...,7-Speed A/T,Black,Black,None reported,"$15,500"
4,Audi,Q3 45 S line Premium Plus,2021,"9,835 mi.",Gasoline,2.0L I4 16V GDI DOHC Turbo,8-Speed Automatic,Glacier White Metallic,Black,None reported,"$34,999"


In [122]:
# Mostrar todas las filas y columnas
pd.set_option('display.max_columns', None)  # Muestra todas las columnas

print(df)

         brand                            model  model_year      milage  \
0         Ford  Utility Police Interceptor Base        2013  51,000 mi.   
1      Hyundai                     Palisade SEL        2021  34,742 mi.   
2        Lexus                    RX 350 RX 350        2022  22,372 mi.   
3     INFINITI                 Q50 Hybrid Sport        2015  88,900 mi.   
4         Audi        Q3 45 S line Premium Plus        2021   9,835 mi.   
...        ...                              ...         ...         ...   
4004   Bentley             Continental GT Speed        2023     714 mi.   
4005      Audi             S4 3.0T Premium Plus        2022  10,900 mi.   
4006   Porsche                           Taycan        2022   2,116 mi.   
4007      Ford                     F-150 Raptor        2020  33,000 mi.   
4008       BMW                     X3 xDrive30i        2020  43,000 mi.   

          fuel_type                                             engine  \
0     E85 Flex Fuel  300.

In [123]:
df['milage'] = df['milage'].str.replace('mi.', '', regex=False).str.replace(',', '.').astype(float)

In [124]:
df['price'] = df['price'].astype(str) \
    .str.replace('$', '', regex=False) \
    .str.replace(',', '', regex=False) \
    .astype(float)

In [125]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4009 entries, 0 to 4008
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   brand         4009 non-null   object 
 1   model         4009 non-null   object 
 2   model_year    4009 non-null   int64  
 3   milage        4009 non-null   float64
 4   fuel_type     3839 non-null   object 
 5   engine        4009 non-null   object 
 6   transmission  4009 non-null   object 
 7   ext_col       4009 non-null   object 
 8   int_col       4009 non-null   object 
 9   accident      3896 non-null   object 
 10  price         4009 non-null   float64
dtypes: float64(2), int64(1), object(8)
memory usage: 344.7+ KB


In [126]:
df['engine'].unique()

array(['300.0HP 3.7L V6 Cylinder Engine Flex Fuel Capability',
       '3.8L V6 24V GDI DOHC', '3.5 Liter DOHC', ...,
       '136.0HP 1.8L 4 Cylinder Engine Gasoline Fuel',
       '270.0HP 2.0L 4 Cylinder Engine Gasoline Fuel',
       '420.0HP 5.9L 12 Cylinder Engine Gasoline Fuel'],
      shape=(1146,), dtype=object)

In [127]:
df['engine_size'] = df['engine'].str.extract(r'(\d.\d+)\s?L').astype(float)

In [128]:
df['engine_type'] = df['engine'].str.extract(r'(V\d|I\d|Electric|Hybrid|Turbo|Diesel)', expand=False)
df['engine_type'] = df['engine_type'].fillna('Unknown')

In [129]:
df['engine_hp'] = df['engine'].str.extract(r'(\d+.?\d*)\s?HP')
df['engine_hp'] = pd.to_numeric(df['engine_hp'], errors='coerce')

In [130]:
df = df.drop(columns=['engine','ext_col','int_col'])

In [131]:
df['accident'] = df['accident'].map({
    'None reported': 0,
    'At least 1 accident or damage reported': 1
}).where(pd.notna(df['accident'])).astype('Int64')

In [135]:
df['accident'] = df['accident'].fillna(0).astype('Int64')

In [136]:
df.head()

,brand,model,model_year,milage,fuel_type,transmission,accident,price,engine_size,engine_type,engine_hp
0,Ford,Utility Police Interceptor Base,2013,51.000,E85 Flex Fuel,6-Speed A/T,1,10300.0,3.7,V6,300.0
1,Hyundai,Palisade SEL,2021,34.742,Gasoline,8-Speed Automatic,1,38005.0,3.8,V6,NaN
2,Lexus,RX 350 RX 350,2022,22.372,Gasoline,Automatic,0,54598.0,3.5,Unknown,NaN
3,INFINITI,Q50 Hybrid Sport,2015,88.900,Hybrid,7-Speed A/T,0,15500.0,3.5,V6,354.0
4,Audi,Q3 45 S line Premium Plus,2021,9.835,Gasoline,8-Speed Automatic,0,34999.0,2.0,I4,NaN


In [133]:
df['accident'].unique()

<IntegerArray>
[1, 0, <NA>]
Length: 3, dtype: Int64

In [134]:
pd.set_option('display.max_columns', None)  # Muestra todas las columnas

print(df)

         brand                            model  model_year   milage  \
0         Ford  Utility Police Interceptor Base        2013   51.000   
1      Hyundai                     Palisade SEL        2021   34.742   
2        Lexus                    RX 350 RX 350        2022   22.372   
3     INFINITI                 Q50 Hybrid Sport        2015   88.900   
4         Audi        Q3 45 S line Premium Plus        2021    9.835   
...        ...                              ...         ...      ...   
4004   Bentley             Continental GT Speed        2023  714.000   
4005      Audi             S4 3.0T Premium Plus        2022   10.900   
4006   Porsche                           Taycan        2022    2.116   
4007      Ford                     F-150 Raptor        2020   33.000   
4008       BMW                     X3 xDrive30i        2020   43.000   

          fuel_type                       transmission  accident     price  \
0     E85 Flex Fuel                        6-Speed A/T   